## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
import sys
sys.path.append("C:/Users/Kompyuter/anaconda3/envs/PythonData/lib/site-packages")


In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import google_key

# Configure gmaps API key
gmaps.configure(api_key=google_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,62.19,80,100,1.01,overcast clouds
1,1,Bambous Virieux,MU,-20.3428,57.7575,79.05,73,40,12.66,scattered clouds
2,2,Sibolga,ID,1.7427,98.7792,78.49,86,84,3.18,heavy intensity rain
3,3,Atuona,PF,-9.8000,-139.0333,78.85,81,20,18.10,few clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.10,72,47,16.60,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))


What is your desired minimum temperature for your trip? 0
What is your desired maximum temperature for your trip? 100


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,62.19,80,100,1.01,overcast clouds
1,1,Bambous Virieux,MU,-20.3428,57.7575,79.05,73,40,12.66,scattered clouds
2,2,Sibolga,ID,1.7427,98.7792,78.49,86,84,3.18,heavy intensity rain
3,3,Atuona,PF,-9.8000,-139.0333,78.85,81,20,18.10,few clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.10,72,47,16.60,light rain
5,5,Colares,PT,38.7992,-9.4469,77.99,50,0,10.36,clear sky
6,6,Kirakira,SB,-10.4544,161.9205,79.25,84,77,5.19,broken clouds
7,7,Ushuaia,AR,-54.8000,-68.3000,42.46,65,75,18.41,broken clouds
8,8,Olafsvik,IS,64.8945,-23.7142,44.13,97,100,12.48,light rain
9,9,Kapaa,US,22.0752,-159.3190,73.72,87,75,14.97,broken clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,62.19,80,100,1.01,overcast clouds
1,1,Bambous Virieux,MU,-20.3428,57.7575,79.05,73,40,12.66,scattered clouds
2,2,Sibolga,ID,1.7427,98.7792,78.49,86,84,3.18,heavy intensity rain
3,3,Atuona,PF,-9.8000,-139.0333,78.85,81,20,18.10,few clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.10,72,47,16.60,light rain
...,...,...,...,...,...,...,...,...,...,...
683,683,Westport,US,41.1415,-73.3579,68.14,47,100,4.00,overcast clouds
684,684,Luwuk,ID,-0.9516,122.7875,78.21,88,98,2.24,overcast clouds
685,685,Itarema,BR,-2.9248,-39.9167,84.45,70,96,7.81,overcast clouds
686,686,Rajanpur,PK,29.1035,70.3250,91.80,5,9,2.84,clear sky


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,62.19,overcast clouds,-33.5906,26.8910,
1,Bambous Virieux,MU,79.05,scattered clouds,-20.3428,57.7575,
2,Sibolga,ID,78.49,heavy intensity rain,1.7427,98.7792,
3,Atuona,PF,78.85,few clouds,-9.8000,-139.0333,
4,Rikitea,PF,78.10,light rain,-23.1203,-134.9692,
5,Colares,PT,77.99,clear sky,38.7992,-9.4469,
6,Kirakira,SB,79.25,broken clouds,-10.4544,161.9205,
7,Ushuaia,AR,42.46,broken clouds,-54.8000,-68.3000,
8,Olafsvik,IS,44.13,light rain,64.8945,-23.7142,
9,Kapaa,US,73.72,broken clouds,22.0752,-159.3190,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Sk

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Port Alfred,ZA,62.19,overcast clouds,-33.5906,26.8910,The Halyards Hotel
1,Bambous Virieux,MU,79.05,scattered clouds,-20.3428,57.7575,Casa Tia Villa
2,Sibolga,ID,78.49,heavy intensity rain,1.7427,98.7792,Hotel Syariah CN Darussalam
3,Atuona,PF,78.85,few clouds,-9.8000,-139.0333,Villa Enata
4,Rikitea,PF,78.10,light rain,-23.1203,-134.9692,People ThankYou


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations.head()

,Lat,Lng
0,-33.5906,26.8910
1,-20.3428,57.7575
2,1.7427,98.7792
3,-9.8000,-139.0333
4,-23.1203,-134.9692


In [16]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [17]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [18]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))